## Setup

In [9]:
import numpy as np
import salabim as sim

In [26]:
import salabim as sim

# Constants
up = 1
down = -1
still = 0

move_time = 2
dooropen_time = 2
doorclose_time = 2

enter_time = 2
exit_time = 1

capacity = 8

topfloor = 3  # 0: Ground floor, 1-3: Employee floors

def run_simulation(seed: int, trace=False, sim_duration=4*3600, warmup=3600) -> list:
    env = sim.Environment(random_seed=seed)
    
    class Visitor(sim.Component):
        def setup(self, from_, to, *args, **kwargs):
            self.fromfloor = floors[from_]
            self.tofloor = floors[to]
            self.direction = getdirection(self.fromfloor, self.tofloor)

        def process(self):
            self.enter(self.fromfloor.visitors)
            if not (self.fromfloor, self.direction) in requests:
                requests[self.fromfloor, self.direction] = self.env.now()
            Elevator.activate()
            self.passivate()

            

    class Elevator(sim.Component):
        def setup(self, capacity):
            self.capacity = capacity
            self.direction = still
            self.floor = floors[0]
            self.visitors = sim.Queue(name="visitors in Elevator")
            self.time_spent_moving = 0  # Initialize the time_spent_moving attribute


        def process(self):
            while True:
                for direction, target_floor in [(up, topfloor), (down, 0)]:
                    for floor in range(self.floor.n, target_floor + direction, direction):
                        current_floor = floors[floor]

                        # Unload passengers if needed
                        if any(visitor.tofloor == current_floor for visitor in self.visitors):
                            self.hold(exit_time)
                            for visitor in list(self.visitors):
                                if visitor.tofloor == current_floor:
                                    visitor.leave(self.visitors)
                                    visitor.activate()

                        # Load passengers if needed
                        if (current_floor, direction) in requests:
                            del requests[current_floor, direction]
                            self.hold(dooropen_time)

                            for visitor in list(current_floor.visitors):
                                if len(self.visitors) < capacity:
                                    visitor.leave(current_floor.visitors)
                                    visitor.enter(self.visitors)

                            self.hold(enter_time)
                            self.hold(doorclose_time)

                        # Move to the next floor
                        if floor != target_floor:
                            self.hold(move_time)
                        self.floor = floors[floor]


        def count_to_floor(self, tofloor):
            n = 0
            for visitor in self.visitors:
                if visitor.tofloor == tofloor:
                    n += 1
            return n


    class Floor:
        def __init__(self, n):
            self.n = n
            self.visitors = sim.Queue(name=f"visitors {n}")

    def getdirection(fromfloor, tofloor):
        return up if fromfloor.n < tofloor.n else down


    # Initialize the simulation environment
    env = sim.Environment(random_seed=42)

    # Initialize the floor and elevator instances
    requests = {}
    floors = {ifloor: Floor(ifloor) for ifloor in range(topfloor + 1)}
    Elevator = Elevator(capacity=capacity, at=0)

    # Generate 100 employees going from the ground floor to random employee floors
    for i in range(100):
        to_floor = sim.random.randint(1, 3)
        from_floor = sim.random.randint(1, 3)
        Visitor(from_=from_floor, to=to_floor, at=i*36)

    # Generate 80 random moves for the 8 working hours (10 moves per hour)
    for i in range(80):
        from_floor, to_floor = sim.random.randint(0, 3), sim.random.randint(0, 3)
        while from_floor == to_floor:
            from_floor, to_floor = sim.random.randint(0, 3), sim.random.randint(0, 3)
        Visitor(from_=from_floor, to=to_floor, at=3600 + i*450)

    env.run(duration=36000)  # Run for 10 hours (3600*10)



    print("Floor    n         length length_of_stay")
    for floor in floors.values():
        print(
            "{:5d}{:5d}{:15.3f}{:15.3f}".format(
                floor.n,
                floor.visitors.length_of_stay.number_of_entries(),
                floor.visitors.length.mean(),
                floor.visitors.length_of_stay.mean(),
            )
        )
        


In [27]:
if __name__ == "__main__":
    run_simulation(seed=42, trace=False, sim_duration=10*3600, warmup=3600)


NameError: name 'Elevators' is not defined

In [38]:
import salabim as sim

# Constants
up = 1
down = -1
still = 0

move_time = 2
dooropen_time = 2
doorclose_time = 2

enter_time = 2
exit_time = 1

capacity = 8

topfloor = 12  # 0: Ground floor, 1-3: Employee floors

id = 1 #persons created in the morning, la

def run_simulation(seed: int, trace=True, sim_duration=4*3600, warmup=3600) -> list:
    env = sim.Environment(random_seed=seed)


    class Person(sim.Component):
        def setup(self, from_, to, *args, **kwargs):
            self.fromfloor = floors[from_]
            self.tofloor = floors[to]
            self.direction = getdirection(self.fromfloor, self.tofloor)

        def process(self):
            self.enter(self.fromfloor.Persons)
            if not (self.fromfloor, self.direction) in requests:
                requests[self.fromfloor, self.direction] = self.env.now()
            self.enter(waitingline) #put itself at tail of the waiting line
            Elevator.activate()
            self.passivate()


    class Elevator(sim.Component):
        def setup(self, capacity):
            self.capacity = capacity
            self.direction = still
            self.floor = floors[0]
            self.Persons = sim.Queue(name="Persons in Elevator")
            self.time_spent_moving = 0  # Initialize the time_spent_moving attribute


        def process(self):
            while True:
                for direction, target_floor in [(up, topfloor), (down, 0)]:
                    for floor in range(self.floor.n, target_floor + direction, direction):
                        current_floor = floors[floor]

                        # Unload passengers if needed
                        if any(Person.tofloor == current_floor for Person in self.Persons):
                            self.hold(exit_time)
                            for Person in list(self.Persons):
                                if Person.tofloor == current_floor:
                                    Person.leave(self.Persons)
                                    Person.activate()

                        # Load passengers if needed
                        if (current_floor, direction) in requests:
                            del requests[current_floor, direction]
                            self.hold(dooropen_time)

                            for Person in list(current_floor.Persons):
                                if len(self.Persons) < capacity:
                                    Person.leave(current_floor.Persons)
                                    self.person = waitingline.pop()
                                    Person.enter(self.Persons)

                            self.hold(enter_time)
                            self.hold(doorclose_time)

                        # Move to the next floor
                        if floor != target_floor:
                            self.hold(move_time)
                        self.floor = floors[floor]


        def count_to_floor(self, tofloor):
            n = 0
            for Person in self.Persons:
                if Person.tofloor == tofloor:
                    n += 1
            return n


    class Floor:
        def __init__(self, n):
            self.n = n
            self.Persons = sim.Queue(name=f"Persons {n}")

    def getdirection(fromfloor, tofloor):
        return up if fromfloor.n < tofloor.n else down


    waitingline = sim.Queue("waitingline")

    # Initialize the simulation environment
    env = sim.Environment(random_seed=seed)

    # Initialize the floor and elevator instances
    requests = {}
    floors = {ifloor: Floor(ifloor) for ifloor in range(topfloor + 1)}
    Elevator = Elevator(capacity=capacity, at=0)

    # Generate 100 employees going from the ground floor to random employee floors
    for i in range(100):
        to_floor = sim.random.randint(1, 3)
        from_floor = sim.random.randint(1, 3)
        Person(from_=from_floor, to=to_floor, at=i*36)

    # Generate 80 random moves for the 8 working hours (10 moves per hour)
    for i in range(80):
        from_floor, to_floor = sim.random.randint(0, topfloor), sim.random.randint(0, topfloor)
        while from_floor == to_floor:
            from_floor, to_floor = sim.random.randint(0, topfloor), sim.random.randint(0, topfloor)
        Person(from_=from_floor, to=to_floor, at=3600 + i*450)

    env.run(duration=36000)  # Run for 10 hours (3600*10)



    print("Floor    n         length length_of_stay")
    for floor in floors.values():
        print(
            "{:5d}{:5d}{:15.3f}{:15.3f}".format(
                floor.n,
                floor.Persons.length_of_stay.number_of_entries(),
                floor.Persons.length.mean(),
                floor.Persons.length_of_stay.mean(),
            )
        )
    waitingline.print_statistics()



In [39]:
if __name__ == "__main__":
    run_simulation(seed=42, trace=False, sim_duration=10*3600, warmup=3600)


Floor    n         length length_of_stay
    0    8          0.007         32.000
    1   46          0.033         25.674
    2   36          0.030         30.500
    3   43          0.035         29.023
    4    2          0.000          6.000
    5    4          0.002         21.000
    6    6          0.006         36.000
    7    7          0.003         16.857
    8    6          0.003         16.000
    9    2          0.001         18.000
   10    3          0.002         20.000
   11    3          0.002         20.667
   12    6          0.005         32.000
Statistics of waitingline at         0    
                                                                     all    excl.zero         zero
-------------------------------------------- -------------- ------------ ------------ ------------
Length of waitingline                        no data

Length of stay in waitingline                entries             172          172            0    
                                